In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('dataset.csv')

In [4]:
df.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


### Use an automated feature selection method to identify the important features in the dataset.

In [10]:
def important_features(df, target):
    return df.corr()[target][:-1].sort_values(ascending=False)

In [12]:
important_features(df, 'target' )

cp          0.433798
thalach     0.421741
slope       0.345877
restecg     0.137230
fbs        -0.028046
chol       -0.085239
trestbps   -0.144931
age        -0.225439
sex        -0.280937
thal       -0.344029
ca         -0.391724
oldpeak    -0.430696
exang      -0.436757
Name: target, dtype: float64

### Create a numerical pipeline that includes the following steps
### Impute the missing values in the numerical columns using the mean of the column value
### Scale the numerical columns using standardisation

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [13]:
from sklearn.pipeline import Pipeline

In [15]:
num_pipe = Pipeline(
steps = [
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
]
)

### Create a categorical pipeline that includes the following steps"
### Impute the missing values in the categorical columns using the most frequent value of the column
### One-hot encode the categorical columns

In [17]:
cat_pipe = Pipeline(
steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
]
)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [45]:
X=df.drop('target', axis=1)
y=df['target']

In [46]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


### Combine the numerical and categorical pipelines using a ColumnTransformer

In [52]:
def num_cat_columns(df):
    num_cols =[]
    cat_cols =[]
    
    for i in df.columns:
        if df[i].dtype in ['int64', 'float64']:
            num_cols.append(i)
        else :
            cat_cols.append(i)
    return num_cols, cat_cols

In [53]:
num_cols, cat_cols = num_cat_columns(X)

In [55]:
preprocessor = ColumnTransformer([
    ('num_pipe', num_pipe, num_cols),
    ('cat_pipe', cat_pipe, cat_cols)
])

In [56]:
from sklearn.model_selection import train_test_split

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [61]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

### Use a Random Forest Classifier to build the final model
### Evaluate the accuracy of the model on the test dataset

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [63]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8131868131868132


### Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [64]:
df = sns.load_dataset('iris')

In [65]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [66]:
from sklearn.preprocessing import LabelEncoder

In [67]:
label = LabelEncoder()

In [68]:
df['species'] = label.fit_transform(df['species'])

In [69]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [70]:
X=df.drop('species', axis=1)
y = df['species']

In [71]:
num_cols, cat_cols = num_cat_columns(X)

In [72]:
preprocessor = ColumnTransformer([
    ('num_pipe', num_pipe, num_cols),
    ('cat_pipe', cat_pipe, cat_cols)
])

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [74]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [75]:
from sklearn.linear_model import LogisticRegression

In [76]:
models = {
    'Random Forest Classifier': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression()
}

In [86]:
def evaluate_model(X_train, y_train, X_test, y_test, models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        
        model.fit(X_train, y_train)
        
        pred = model.predict(X_test)
        
        score = accuracy_score(y_test, pred)
        
        report[list(models.keys())[i]] = score
        
    return report

In [87]:
evaluate_model(X_train, y_train, X_test, y_test, models)

{'Random Forest Classifier': 1.0, 'Logistic Regression': 1.0}

In [89]:
from sklearn.ensemble import VotingClassifier
clf1 = LogisticRegression(random_state=42)
clf2 = RandomForestClassifier(random_state=42)

# Create a Voting Classifier with hard voting (majority rule)
voting_clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)], voting='hard')

# Fit the Voting Classifier on the training data
voting_clf.fit(X_train, y_train)

# Make predictions using the Voting Classifier
y_pred = voting_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of Voting Classifier: {accuracy:.2f}')

Accuracy of Voting Classifier: 1.00
